In [38]:
import cv2
import numpy as np
import random
random.seed(0)

In [2]:
def repattack_DC128_ACmsb01_baseline (image_name):
    img_file = image_name + '.tif'
    intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    height = np.shape(intensities)[0]
    width = np.shape(intensities)[1]
    imf = np.float32(intensities)

    filename = image_name + '_gray.tif'
    cv2.imwrite(filename, intensities)

    eve_msb0_intensities = np.zeros([height,width])
    eve_msb1_intensities = np.zeros([height,width])

    prior = 0
    prior_dc0 = 0
    prior_dc1 = 0
    dDCss = []
    k = 0
    for i in range(0,int(height/8)):
        for j in range(0,int(width/8)):
            blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
            shifted_blk = blk - 128 #level_shift for dct
            dct = cv2.dct(shifted_blk)
            nq_coef = np.round(dct)
            nq_coef_ac = np.delete(nq_coef, [0,0])
            nq_coef_dc = nq_coef[0,0]

            eve_ac_msb0 = nq_coef_ac.copy()
            eve_ac_msb1 = nq_coef_ac.copy()

            # AC MSB
            for m in range(np.size(nq_coef_ac)):
                if nq_coef_ac[m] < 0:
                    eve_ac_msb0[m] =  nq_coef_ac[m]

                    size_ac = np.floor(np.log2(abs(nq_coef_ac[m])))+1
                    deltaS = 2**size_ac - 1 + 2**(size_ac-1)
                    eve_ac_msb1[m] = nq_coef_ac[m] + deltaS
                if nq_coef_ac[m] > 0:
                    eve_ac_msb1[m] =  nq_coef_ac[m]

                    size_ac = np.floor(np.log2(abs(nq_coef_ac[m])))+1
                    deltaS = 2**size_ac - 1 + 2**(size_ac-1)
                    eve_ac_msb0[m] = nq_coef_ac[m] - deltaS

            msb0_dc = 128
            msb1_dc = 128

            eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, msb0_dc)
            eve_msb0_coef = np.reshape(eve_msb0_coef1,[8,8])

            eve_msb0_blk1 = cv2.idct(eve_msb0_coef)
            eve_msb0_blk2 = eve_msb0_blk1+128 # level_shift after idct
            eve_msb0_blk = np.round(eve_msb0_blk2)  
            eve_msb0_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb0_blk


            eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, msb1_dc)
            eve_msb1_coef = np.reshape(eve_msb1_coef1,[8,8])

            eve_msb1_blk1 = cv2.idct(eve_msb1_coef)
            eve_msb1_blk2 = eve_msb1_blk1+128 # level_shift after idct
            eve_msb1_blk = np.round(eve_msb1_blk2)  
            eve_msb1_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb1_blk

    # MAJOR MODIFICATION: negative to 0 and 255< to 255
    # after Modification ################################################

    # MSB0 and MSB 1 for dDCs
    eve_msb0_modified = eve_msb0_intensities.copy()
    eve_msb0_modified[eve_msb0_modified<0] = 0
    eve_msb0_modified[eve_msb0_modified>255] = 255

    filename = image_name + '_msb0_DC128_acMSB_OrigBC.png'
    cv2.imwrite(filename, eve_msb0_modified)
    cv2.waitKey(1000)


    eve_msb1_modified = eve_msb1_intensities.copy()
    eve_msb1_modified[eve_msb1_modified<0] = 0
    eve_msb1_modified[eve_msb1_modified>255] = 255

    filename = image_name + '_msb1_DC128_acMSB_OrigBC.png'
    cv2.imwrite(filename, eve_msb1_modified)
    cv2.waitKey(1000)

In [3]:
image_names = ['lena', 'cam', 'man', 'liv', 'gol']
for image_name in image_names:
    repattack_DC128_ACmsb01_baseline(image_name)   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [43]:
def repattack_DC128_rACmsb_baseline (image_name):
    img_file = image_name + '.tif'
    intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    height = np.shape(intensities)[0]
    width = np.shape(intensities)[1]
    imf = np.float32(intensities)

    filename = image_name + '_gray.tif'
    cv2.imwrite(filename, intensities)

    eve_rmsb_intensities = np.zeros([height,width])

    prior = 0
    prior_dc0 = 0
    prior_dc1 = 0
    dDCss = []
    k = 0
    for i in range(0,int(height/8)):
        for j in range(0,int(width/8)):
            blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
            shifted_blk = blk - 128 #level_shift for dct
            dct = cv2.dct(shifted_blk)
            nq_coef = np.round(dct)
            nq_coef_ac = np.delete(nq_coef, [0,0])
            nq_coef_dc = nq_coef[0,0]

            eve_ac_rmsb = nq_coef_ac.copy()
            # AC MSB
            for m in range(np.size(nq_coef_ac)):
                rmsb = random.randint(0,1)
                if rmsb == 0:
                    if nq_coef_ac[m] < 0:
                        eve_ac_rmsb[m] =  nq_coef_ac[m]
                    if nq_coef_ac[m] > 0:
                        size_ac = np.floor(np.log2(abs(nq_coef_ac[m])))+1
                        deltaS = 2**size_ac - 1 + 2**(size_ac-1)
                        eve_ac_rmsb[m] = nq_coef_ac[m] - deltaS

                if rmsb == 1:
                    if nq_coef_ac[m] > 0:
                        eve_ac_rmsb[m] =  nq_coef_ac[m]
                    if nq_coef_ac[m] < 0:
                        size_ac = np.floor(np.log2(abs(nq_coef_ac[m])))+1
                        deltaS = 2**size_ac - 1 + 2**(size_ac-1)
                        eve_ac_rmsb[m] = nq_coef_ac[m] + deltaS        


            const_dc = 128

            eve_rmsb_coef1 = np.insert(eve_ac_rmsb, 0, const_dc)
            eve_rmsb_coef = np.reshape(eve_rmsb_coef1,[8,8])

            eve_rmsb_blk1 = cv2.idct(eve_rmsb_coef)
            eve_rmsb_blk2 = eve_rmsb_blk1+128 # level_shift after idct
            eve_rmsb_blk = np.round(eve_rmsb_blk2)  
            eve_rmsb_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_rmsb_blk


    # MAJOR MODIFICATION: negative to 0 and 255< to 255
    # after Modification ################################################

    # MSB0 and MSB 1 for dDCs
    eve_rmsb_modified = eve_rmsb_intensities.copy()
    eve_rmsb_modified[eve_rmsb_modified<0] = 0
    eve_rmsb_modified[eve_rmsb_modified>255] = 255

    filename = image_name + '_rmsb_DC128_acMSB_OrigBC.png'
    cv2.imwrite(filename, eve_rmsb_modified)
    cv2.waitKey(1000)


In [44]:
image_names = ['lena', 'cam', 'man', 'liv', 'gol']
for image_name in image_names:
    repattack_DC128_rACmsb_baseline(image_name)  

0